# Object Detection: People, Bikes, and Cars
Below we will do the following:
1. Setup the training environment.
2. Load images of cars, bikes and people and their corresponding bounding boxes.
2. Build an object detection model for identifying the location of cars, bikes and people in images.
3. Convert the model to CoreML and download it.

The example is based on [Turi Create's Object Detection model](https://github.com/apple/turicreate/tree/master/userguide/object_detection).

## Environment Setup
Below we use pip to install `turicreate`.

In [0]:
!pip install turicreate==5.4

## Data Preparation and Model Training
The data for this example are 1096 images: 420 cars, 365 bikes, and 311 people. You can find the original dataset [here](https://lear.inrialpes.fr/people/marszalek/data/ig02/).

Once loaded, the data is randomly split into train and test sets, where 80% of the data is used for training and 20% is used for model evaluation (if you desire). Training this model with a GPU is much faster than CPU time. By default, this runtime environment will use Python 3 CPU backend instance.

We use CPU here because exporting TuriCreate's Object Detection model to coreml is [a known issue](https://github.com/apple/turicreate/issues/1249). We are continuing to work through training + exporting with GPU on Google Colab.

In [0]:
# Import libraries
import tarfile
import urllib
import turicreate as tc

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
## Load the data
data =  tc.SFrame('ig02.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

In [0]:
train_data.head()

In [0]:
# Create an object detection model
model = tc.object_detector.create(
    dataset=train_data,
    max_iterations=1 # to improve model accuracy, remove the max_iterations argument and TuriCreate will tune to an appropriate number
)

# Object Detector Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.object_detector.create.html#turicreate.object_detector.create

In [0]:
model.evaluate(test_data)

## Model Export and Download
We convert the model to CoreML format so that it can run on an iOS device. Then we download it locally so it can be delivered to your apps with Skafos.

In [0]:
# Specify the CoreML model name
model_name = 'ObjectDetection'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 


In [0]:
# Download the model you just trained!! This may take a few moments
from google.colab import files
files.download(coreml_model_name)